## MODIS rolling 16-day max NDVI composite

`ImageCollection.map_window` lets you slide a "window" along an ImageCollection, image by image. Here, we use it to composite the previous 16 days of MODIS imagery, an effective way of correcting for clouds when looking at NDVI.

In [ ]:
import descarteslabs.workflows as wf

In [14]:
wf.map.center = [36.50963615733049, -105.89996337890626]
wf.map.zoom = 8

[36.50963615733049, -105.89996337890626]

In [13]:
wf.map


`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jupyter notebook.)
To install for JupyterLab, run this in a cell:
    !jupyter labextension install jupyter-leaflet @jupyter-widgets/jupyterlab-manager
To install for plain Jupyter Notebook, run this in a cell:
    !jupyter nbextension enable --py --sys-prefix ipyleaflet
Then, restart Jupyter and re-run this notebook.


In [3]:
modis = wf.ImageCollection.from_id('modis:09:v2', start_datetime="2019-01-01", end_datetime="2019-08-01")

In [4]:
ndvi = wf.normalized_difference(modis.pick_bands('nir'), modis.pick_bands('red'))

In [10]:
ndvi_16 = (
    ndvi
    .groupby(lambda img: img.properties['date'] // wf.Timedelta(days=1))
        .max(axis="images")
    # ^ first, flatten multiple images from the same day (at scene boundaries) into 1,
    # so we're always getting exactly 16 days
    .map_window(
        lambda back, img, fwd: back.max(axis="images"),
        back=16
    )
    # ^ max-composite the previous 16 days
)

# ndvi_16 is an ImageCollection; each Image is one of our 16-day composites

In [8]:
ndvi_16[0].visualize('jan NDVI', scales=[[0, 1]], colormap="viridis")
# jan 1 - 16 composite

In [9]:
ndvi_16[-1].visualize('july NDVI', scales=[[0, 1]], colormap="viridis")
# july 16 - aug 1 composite